In [1]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pyomo
import pyomo.opt
import pyomo.environ as pe

from src.parsers import HMParser

In [2]:
# Data parsing

data = HMParser(file_path='data/EC_V4.xlsx', ec_id=1)
data.parse()

In [3]:
# Auxiliary function to convert numpy arrays to dictionaries

def convert_to_dictionary(a):
    temp_dictionary = {}

    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1)] = a[dim0, dim1]

    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1)] = a[dim0]

    return temp_dictionary

In [42]:
# Create model and attribute sets

model = pe.ConcreteModel()

# Set default behaviour
default_behaviour = pe.Constraint.Skip

# Variable iterators
model.t = pe.Set(initialize=np.arange(1, data.generator['p_forecast'].shape[1] + 1),
                 doc='Time periods')
model.cs = pe.Set(initialize=np.arange(1, data.charging_station['p_charge_limit'].shape[0] + 1),
                  doc='Number of charging stations')

In [43]:
# Create generator properties

# Sets and Parameters
model.gen = pe.Set(initialize=np.arange(1, data.generator['p_forecast'].shape[0] + 1),
                   doc='Number of generators')
model.genType = pe.Param(model.gen,
                         initialize=convert_to_dictionary(data.generator['type_generator'].astype(int)),
                         doc='Types of generators')
model.genMin = pe.Param(model.gen,
                        initialize=convert_to_dictionary(data.generator['p_min']),
                        doc='Minimum power generation')
model.genMax = pe.Param(model.gen, model.t,
                        initialize=convert_to_dictionary(data.generator['p_forecast']),
                        doc='Forecasted power generation')

# Variables
model.genActPower = pe.Var(model.gen, model.t, within=pe.NonNegativeReals, initialize=0,
                           doc='Active power generation')
model.genExcPower = pe.Var(model.gen, model.t, within=pe.NonNegativeReals, initialize=0,
                           doc='Excess power generation')
model.genXo = pe.Var(model.gen, model.t, within=pe.Binary, initialize=0,
                     doc='Generation on/off')

# Constraints
def _genActMaxEq(m, g, t):
    if m.genType[g] == 1:
        return m.genActPower[g, t] <= m.genMax[g, t]
    elif m.genType[g] == 2:
        return m.genActPower[g, t] + m.genExcPower[g, t] == m.genMax[g, t]
    return default_behaviour
model.genActMaxEq = pe.Constraint(model.gen, model.t, rule=_genActMaxEq,
                                  doc='Maximum active power generation')
def _genActMinEq(m, g, t):
    if m.genType[g] == 1:
        return m.genActPower[g, t] >= m.genMin[g] * m.genXo[g, t]
    return default_behaviour
model.genActMinEq = pe.Constraint(model.gen, model.t, rule=_genActMinEq,
                                  doc='Minimum active power generation')

In [44]:
# Create load properties

# Sets and Parameters
model.loads = pe.Set(initialize=np.arange(1, data.load['p_forecast'].shape[0] + 1),
                    doc='Number of loads')
model.loadMax = pe.Param(model.loads, model.t,
                         initialize=convert_to_dictionary(data.load['p_forecast'] * 5),
                         doc='Forecasted power consumption')
model.loadRedMax = pe.Param(model.loads, model.t,
                            initialize=convert_to_dictionary(data.load['p_forecast'] * 0.5),
                            doc='Maximum power reduction')
model.loadCutMax = pe.Param(model.loads, model.t,
                            initialize=convert_to_dictionary(data.load['p_forecast'] * 0.5),
                            doc='Maximum power cut')

# Variables
model.loadRedActPower = pe.Var(model.loads, model.t, within=pe.NonNegativeReals, initialize=0,
                               doc='Active power reduction')
model.loadCutActPower = pe.Var(model.loads, model.t, within=pe.NonNegativeReals, initialize=0,
                               doc='Active power cut')
model.loadENS = pe.Var(model.loads, model.t, within=pe.NonNegativeReals, initialize=0,
                       doc='Energy Not Supplied')
model.loadXo = pe.Var(model.loads, model.t, within=pe.Binary, initialize=0,
                      doc='Load on/off')

# Constraints
def _loadRedActEq(m, l, t):
    return m.loadRedActPower[l, t] <= m.loadRedMax[l, t]
model.loadReactEq = pe.Constraint(model.loads, model.t, rule=_loadRedActEq,
                                  doc='Maximum active power reduction')

def _loadCutActEq(m, l, t):
    return m.loadCutActPower[l, t] == m.loadCutMax[l, t] * m.loadXo[l, t]
model.loadCutActEq = pe.Constraint(model.loads, model.t, rule=_loadCutActEq,
                                   doc='Maximum active power cut')

def _loadENSEq(m, l, t):
    return m.loadENS[l, t] + m.loadRedActPower[l, t] + m.loadCutActPower[l, t] <= m.loadMax[l, t]
model.loadENSEq = pe.Constraint(model.loads, model.t, rule=_loadENSEq,
                                doc='Maximum energy not supplied')

In [45]:
# Create storage properties

# Sets and Parameters
model.stor = pe.Set(initialize=np.arange(1, data.storage['p_charge_limit'].shape[0] + 1),
                    doc='Number of storage units')
model.storDchMax = pe.Param(model.stor, model.t,
                            initialize=convert_to_dictionary(data.storage['p_discharge_limit']),
                            doc='Maximum discharging power')
model.storChMax = pe.Param(model.stor, model.t,
                           initialize=convert_to_dictionary(data.storage['p_charge_limit']),
                           doc='Maximum charging power')
model.storMax = pe.Param(model.stor,
                         initialize=convert_to_dictionary(data.storage['energy_capacity']),
                         doc='Maximum energy capacity')
model.storMin = pe.Param(model.stor,
                         initialize=convert_to_dictionary(data.storage['energy_min_percentage']),
                         doc='Minimum energy capacity')
model.storStart = pe.Param(model.stor,
                           initialize=convert_to_dictionary(data.storage['initial_state']),
                           doc='Starting energy capacity')
model.storDchEff = pe.Param(model.stor,
                            initialize=convert_to_dictionary(data.storage['discharge_efficiency']),
                            doc='Discharging efficiency')
model.storChEff = pe.Param(model.stor,
                           initialize=convert_to_dictionary(data.storage['charge_efficiency']),
                           doc='Charging efficiency')

# Variables
model.storState = pe.Var(model.stor, model.t, within=pe.NonNegativeReals, initialize=0,
                         doc='State of charge')
model.storCharge = pe.Var(model.stor, model.t, within=pe.NonNegativeReals, initialize=0,
                          doc='Charging power')
model.storDischarge = pe.Var(model.stor, model.t, within=pe.NonNegativeReals, initialize=0,
                             doc='Discharging power')
model.storRelax = pe.Var(model.stor, model.t, within=pe.NonNegativeReals, initialize=0,
                         doc='Relaxation variable')
model.storChXo = pe.Var(model.stor, model.t, within=pe.Binary, initialize=0,
                        doc='Charging on/off')
model.storDchXo = pe.Var(model.stor, model.t, within=pe.Binary, initialize=0,
                         doc='Discharging on/off')

# Constraints
def _storDchRateEq(m, s, t):
    return m.storDischarge[s, t] <= m.storDchMax[s, t] * m.storDchXo[s, t]
model.storDchRateEq = pe.Constraint(model.stor, model.t, rule=_storDchRateEq,
                                    doc='Maximum discharging rate')

def _storChRateEq(m, s, t):
    return m.storCharge[s, t] <= m.storChMax[s, t] * m.storChXo[s, t]
model.storChRateEq = pe.Constraint(model.stor, model.t, rule=_storChRateEq,
                                   doc='Maximum charging rate')

def _storMaxEq(m, s, t):
    return m.storState[s, t] <= m.storMax[s]
model.storMaxEq = pe.Constraint(model.stor, model.t, rule=_storMaxEq,
                                doc='Maximum energy capacity')

def _storRelaxEq(m, s, t):
    return m.storState[s, t] >= m.storMax[s] * m.storMin[s]  - m.storRelax[s, t]
model.storRelaxEq = pe.Constraint(model.stor, model.t, rule=_storRelaxEq,
                                  doc='Relaxation variable')

def _storBalanceEq(m, s, t):
    if t == 1:
        return m.storState[s, t] == m.storMax[s] * m.storStart[s] + m.storCharge[s, t] * m.storChEff[s] - m.storDischarge[s, t] / m.storDchEff[s]
    elif t > 1:
        return m.storState[s, t] == m.storState[s, t - 1] + m.storCharge[s, t] * m.storChEff[s] - m.storDischarge[s, t] / m.storDchEff[s]
    return default_behaviour
model.storBalanceEq = pe.Constraint(model.stor, model.t, rule=_storBalanceEq,
                                    doc='Energy balance')

def _storChDchEq(m, s, t):
    return m.storChXo[s, t] + m.storDchXo[s, t] <= 1
model.storChDchEq = pe.Constraint(model.stor, model.t, rule=_storChDchEq,
                                  doc='Charging and discharging are mutually exclusive')

In [46]:
# Create V2G properties

# Sets and Parameters
model.v2g = pe.Set(initialize=np.arange(1, data.vehicle['p_charge_max'].shape[0] + 1),
                   doc='Number of EVs')
model.v2gDchMax = pe.Param(model.v2g, model.t,
                           initialize=convert_to_dictionary((data.vehicle['p_discharge_max'] *
                                                             data.vehicle['schedule'].transpose()).transpose()),
                           doc='Maximum scheduled discharging power')
model.v2gChMax = pe.Param(model.v2g, model.t,
                          initialize=convert_to_dictionary((data.vehicle['p_charge_max'] *
                                                            data.vehicle['schedule'].transpose()).transpose()),
                          doc='Maximum scheduled charging power')
model.v2gDchEff = pe.Param(model.v2g,
                           initialize=convert_to_dictionary(data.vehicle['discharge_efficiency']),
                           doc='Discharging efficiency')
model.v2gChEff = pe.Param(model.v2g,
                          initialize=convert_to_dictionary(data.vehicle['charge_efficiency']),
                          doc='Charging efficiency')
model.v2gMax = pe.Param(model.v2g,
                        initialize=convert_to_dictionary(data.vehicle['e_capacity_max']),
                        doc='Maximum energy capacity')
model.v2gMin = pe.Param(model.v2g,
                        initialize=convert_to_dictionary(data.vehicle['e_capacity_max'] * data.vehicle['min_technical_soc']),
                        doc='Minimum energy capacity')
model.v2gSchedule = pe.Param(model.v2g, model.t,
                             initialize=convert_to_dictionary(data.vehicle['schedule']),
                             doc='Vehicle schedule')
model.v2gScheduleArrivalSOC = pe.Param(model.v2g, model.t,
                                       initialize=convert_to_dictionary(data.vehicle['schedule_arrival_soc']),
                                       doc='Vehicle schedule arrival SOC')
model.v2gScheduleDepartureSOC = pe.Param(model.v2g, model.t,
                                         initialize=convert_to_dictionary(data.vehicle['schedule_departure_soc']),
                                         doc='Vehicle schedule required')

# Variables
model.v2gCharge = pe.Var(model.v2g, model.t, within=pe.NonNegativeReals, initialize=0,
                         doc='Charging power')
model.v2gDischarge = pe.Var(model.v2g, model.t, within=pe.NonNegativeReals, initialize=0,
                            doc='Discharging power')
model.v2gState = pe.Var(model.v2g, model.t, within=pe.NonNegativeReals, initialize=0,
                        doc='State of charge')
model.v2gRelax = pe.Var(model.v2g, model.t, within=pe.NonNegativeReals, initialize=0,
                        doc='Relaxation variable')
model.v2gChXo = pe.Var(model.v2g, model.t, within=pe.Binary, initialize=0,
                       doc='Charging on/off')
model.v2gDchXo = pe.Var(model.v2g, model.t, within=pe.Binary, initialize=0,
                        doc='Discharging on/off')

# Constraints
def _v2gDchRateEq(m, v, t):
    return m.v2gDischarge[v, t] <= m.v2gDchMax[v, t] * m.v2gDchXo[v, t]
model.v2gDchRateEq = pe.Constraint(model.v2g, model.t, rule=_v2gDchRateEq,
                                   doc='Maximum discharging rate')

def _v2gChRateEq(m, v, t):
    return m.v2gCharge[v, t] <= m.v2gChMax[v, t] * m.v2gChXo[v, t]
model.v2gChRateEq = pe.Constraint(model.v2g, model.t, rule=_v2gChRateEq,
                                  doc='Maximum charging rate')

def _v2gMaxEq(m, v, t):
    return m.v2gState[v, t] <= m.v2gMax[v]
model.v2gMaxEq = pe.Constraint(model.v2g, model.t, rule=_v2gMaxEq,
                               doc='Maximum energy capacity')

def _v2gRelaxEq(m, v, t):
    if m.v2gSchedule[v, t] == 1:
        return m.v2gState[v, t] >= m.v2gMin[v] - m.v2gRelax[v, t]
    elif t < m.t.last():
        if (m.v2gSchedule[v, t] == 1) & (m.v2gSchedule[v, t + 1] == 0) & (m.v2gScheduleDepartureSOC[v, t] == 0):
            return m.v2gState[v, t] >= m.v2gMax[v] - m.v2gRelax[v, t]
    elif (m.v2gSchedule[v, t] == 1) & (m.v2gScheduleDepartureSOC[v, t] == 0) & (m.t == m.t.last()):
        return m.v2gState[v, t] >= m.v2gMax[v] - m.v2gRelax[v, t]
    return default_behaviour
model.v2gRelaxEq = pe.Constraint(model.v2g, model.t, rule=_v2gRelaxEq,
                                 doc='Relaxation variable')

def _v2gStateEq(m, v, t):
    if m.v2gSchedule[v, t] == 0: # If vehicle is not scheduled
        return m.v2gState[v, t] == 0
    elif (m.v2gSchedule[v, t] == 1) & (t == 1): # If vehicle is scheduled and it is the first time step
        return m.v2gState[v, t] == m.v2gState[v, t] + m.v2gCharge[v, t] * m.v2gChEff[v] - m.v2gDischarge[v, t] / m.v2gDchEff[v]
    elif t > 1: # If not the first time step
        if (m.v2gSchedule[v, t - 1] == 1) & (m.v2gSchedule[v, t] == 1): # If was and is currently connected
            return m.v2gState[v, t] == m.v2gState[v, t - 1] + m.v2gCharge[v, t] * m.v2gChEff[v] - m.v2gDischarge[v, t] / m.v2gDchEff[v]
        elif (m.v2gSchedule[v, t - 1] == 0) & (m.v2gSchedule[v, t] == 1): # If became connected
            return m.v2gState[v, t] == m.v2gScheduleArrivalSOC[v, t] + m.v2gCharge[v, t] * m.v2gChEff[v] - m.v2gDischarge[v, t] / m.v2gDchEff[v]
    return default_behaviour
model.v2gStateEq = pe.Constraint(model.v2g, model.t, rule=_v2gStateEq,
                                 doc='State of charge')

def _v2gChDchEq(m, v, t):
    return m.v2gCharge[v, t] + m.v2gDischarge[v, t] <= 1
model.v2gChDchEq = pe.Constraint(model.v2g, model.t, rule=_v2gChDchEq,
                                 doc='Charging and discharging cannot occur simultaneously')